In [ ]:
import cv2 as cv
import numpy as np
import keras
import dlib
from imutils.video import FPS

In [ ]:
confThreshold = 0.5  #Confidence threshold
nmsThreshold = 0.4   #Non-maximum suppression threshold
inpWidth = 416       #Width of network's input image
inpHeight = 416      #Height of network's input image

In [ ]:
classesFile = "coco.names";
classes = None
with open(classesFile, 'rt') as f:
    classes = [line.strip() for line in f.readlines()]

#print(classes)

# Give the configuration and weight files for the model and load the network using them.
modelConfiguration = "yolov3.cfg";
modelWeights = "yolov3.weights";

net = cv.dnn.readNetFromDarknet(modelConfiguration, modelWeights)
net.setPreferableBackend(cv.dnn.DNN_BACKEND_OPENCV)
net.setPreferableTarget(cv.dnn.DNN_TARGET_CPU)

In [ ]:
ln = net.getLayerNames()
ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]
print(ln)

def drawPred(classId, conf, left, top, right, bottom):
    cv.rectangle(frame, (left, top), (right, bottom), (255, 178, 50), 3)
    label = '%.2f'%conf
    if classes:
        assert(classId < len(classes))
        label = '%s:%s'%(classes[classId], label)
    
    labelSize, baseline = cv.getTextSize(label, cv.FONT_HERSHEY_SIMPLEX, 0.5, 1)
    top = max(top, labelSize[1])
    cv.rectangle(frame, (left, top - round(1.5*labelSize[1])), (left + round(1.5*labelSize[0]), top + baseline), 
                 (255,255,255), cv.FILLED)
    cv.putText(frame, label, (left, top), cv.FONT_HERSHEY_SIMPLEX, 0.75, (0, 0, 0))

def postprocess(frame, outs):
    frameHeight = frame.shape[0]
    frameWidth = frame.shape[1]
    
    classIds = []
    confidences = []
    boxes = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            classId = np.argmax(scores)
            confidence = scores[classId]
            if confidence > confThreshold:
                center_x = int(detection[0]*frameWidth)
                center_y = int(detection[1]*frameHeight)
                width = int(detection[2] * frameWidth)
                height = int(detection[3] * frameHeight)
                left = int(center_x - width / 2)
                top = int(center_y - height / 2)
                classIds.append(classId)
                confidences.append(float(confidence))
                boxes.append([left, top, width, height])
    indices = cv.dnn.NMSBoxes(boxes, confidences, confThreshold, nmsThreshold)
    for i in indices:
        i = i[0]
        box = boxes[i]
        left = box[0]
        top = box[1]
        width = box[2]
        height = box[3]
        drawPred(classIds[i], confidences[i], left, top, left + width, top + height)

In [ ]:
cap = cv.VideoCapture("C:/Users/shubh/Downloads/project_video.mp4")
writer = None
trackers = []
labels = []

fps = FPS().start()

while cv.waitKey(1) < 0:
    ret, frame = cap.read()
    if not ret:
        cv.waitKey(3000)
        cap.release()
        break
    
    rgb = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
    if len(trackers) == 0:
        blob = cv.dnn.blobFromImage(frame, 1/255.0, (inpHeight, inpWidth), [0, 0, 0], swapRB=True, crop=False)
        net.setInput(blob)
        detections = net.forward(ln)
        print(detections)
        
        boxes = []
        confidences = []
        classIDs = []

        
        for output in detections:
            for detection in output:
                scores = detection[5:]
                classID = np.argmax(scores)
                confidence = scores[classID]
                if confidence > confThreshold:
                    box = detection[0:4] * np.array([inpWidth, inpHeight, inpWidth, inpHeight])
                    (centerX, centerY, width, height) = box.astype("int")
                    
                    x = int(centerX - (width/2))
                    y = int(centerY - (height/2))
                    
                    boxes.append([x, y, int(width), int(height)])
                    confidences.append(float(confidence))
                    classIDs.append(classID)
                    
        idxs = cv.dnn.NMSBoxes(boxes, confidences, 0.5, 0.3)
        if len(idxs) > 0:
            for i in idxs.flatten():
                (x, y) = (boxes[i][0], boxes[i][1])
                (w, h) = (boxes[i][2], boxes[i][3])
                
                if classes[classIDs[i]] == "car":
                    t = dlib.correlation_tracker()
                    rect = dlib.rectangle(x, y, x + w, y + h)
                    t.start_track(rgb, rect)
                    
                    labels.append(label)
                    trackers.append(t)
                
                cv.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                cv.putText(frame, classes[classIDs[i]], (x, y - 5), cv.FONT_HERSHEY_SIMPLEX, 0.45, (0, 255, 0), 2)
                
    else:
        for (t, l) in zip(trackers, labels):
            t.update(rgb)
            pos = t.get_position()
            
            startX = int(pos.left())
            startY = int(pos.top())
            endX = int(pos.right())
            endY = int(pos.bottom())
            
            cv.rectangle(frame, (startX, startY), (endX, endY), (0, 255, 0), 2)
            cv.putText(frame, label, (startX, startY - 15), cv.FONT_HERSHEY_SIMPLEX, 0.45, (0, 255, 0), 2)
            
    cv.imshow("Frame", frame)
    key = cv.waitKey(1) & 0xFF
    
    if key == ord("q"):
        break
    
    fps.update()

    
fps.stop()
print("[INFO] elapsed time: {:.2f}".format(fps.elapsed()))
print("[INFO] approx. FPS: {:.2f}".format(fps.fps()))
                
#postprocess(frame, outs)
    
#    t, _ = net.getPerfProfile()
#    label = 'Inference time: %.2f ms' % (t * 1000.0 / cv.getTickFrequency())
#    cv.putText(frame, label, (0, 15), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255))
#    cv.imshow("Frame", frame)
    
cv.destroyAllWindows()
cap.release()